In [10]:
import pandas as pd
import numpy as np
import logging
from tqdm.auto import tqdm

import country_converter as coco
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

In [3]:
#загрузим все таблицы
#рейтинг городов по безопасности
security_cities_rating = pd.read_excel('parsing_tables/safety_cities_index.xlsx', header=None)
#рейтинг стран по уровню человеческого развития
human_dev_rating = pd.read_csv('parsing_tables/human_dev_index.csv')
#рейтинг городов по уровню качества жизни
quality_of_life_rating = pd.read_csv('parsing_tables/quality_of_living_city_index.csv')
#рейтинг стран по уровню демократии
democracy_rating = pd.read_csv('parsing_tables/democracy_index.csv')


In [4]:
#посмотрим на таблицы, которые у нас есть
print(security_cities_rating.head(3))
print('-----------------------')
print(human_dev_rating.head(3))
print('-----------------------')
print(quality_of_life_rating.head(3))
print('-----------------------')
print(democracy_rating.head(3))


   0           1     2   3           4     5   6          7     8   9   \
0   1  Copenhagen  82.4   1      Sydney  83.2   1      Tokyo  87.7   1   
1   2     Toronto  82.2   2   Singapore  82.8   2  Singapore  84.1   2   
2   3   Singapore  80.7   3  Copenhagen  82.2   3  Hong Kong  84.0   3   

           10    11  12          13    14  15             16    17  
0   Hong Kong  93.4   1  Copenhagen  86.4   1     Wellington  91.7  
1   Singapore  92.1   2   Amsterdam  80.5   2        Toronto  90.3  
2  Copenhagen  89.0   3   Frankfurt  80.3   3  Washington DC  87.6  
-----------------------
   rank      country    hdi average_annual_growth
0     1  Switzerland  0.962                 0.19%
1     2       Norway  0.961                 0.19%
2     3      Iceland  0.959                 0.56%
-----------------------
   rank       city      country
0     1     Vienna      Austria
1     2     Zürich  Switzerland
2     3  Vancouver       Canada
-----------------------
   rank      country     re

### Работа с таблицей quality_of_life_rating

1. Перевод названий стран на английский
2. Меняем колонки местами



In [ ]:
#переведем некоторые страны на английский
l = list(quality_of_life_rating.city)
l[l.index('Zürich')] = 'Zurich'
l[l.index('Düsseldorf')] = 'Dusseldorf'
l[l.index('Montréal')] = 'Montreal'
l[l.index('Pointe-à-Pitre')] = 'Pointe-a-Pitre'
l[l.index('San José')] = 'San Jose'
l[l.index('Asunción')] = 'Asuncion'
l[l.index('São Paulo')] = 'Sao Paulo'
l[l.index('Bogotá')] = 'Bogota'
quality_of_life_rating['city_eng'] = np.array(l)

In [ ]:
#переставим местами некоторые колонки
quality_of_life_rating = quality_of_life_rating.reindex(columns=['rank', 'city', 'city_eng', 'country'])

In [ ]:
#проверяем готовую таблицу
quality_of_life_rating.head(3)

,rank,city,city_eng,country
0,1,Vienna,Vienna,Austria
1,2,Zürich,Zurich,Switzerland
2,3,Vancouver,Vancouver,Canada


### Работа с таблицей security_cities_rating


1. Убираем лишние колонки
2. Делаем индексы по возрастанию без задвоений
3. Новые названия для колонок
4. Находим страны где располагаются города

In [5]:
#выводим таблицу
security_cities_rating.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,Copenhagen,82.4,1,Sydney,83.2,1,Tokyo,87.7,1,Hong Kong,93.4,1,Copenhagen,86.4,1,Wellington,91.7
1,2,Toronto,82.2,2,Singapore,82.8,2,Singapore,84.1,2,Singapore,92.1,2,Amsterdam,80.5,2,Toronto,90.3
2,3,Singapore,80.7,3,Copenhagen,82.2,3,Hong Kong,84.0,3,Copenhagen,89.0,3,Frankfurt,80.3,3,Washington DC,87.6


In [ ]:
#удаляем ненужные колонки
security_cities_rating = security_cities_rating.drop([3, 6 , 9, 12, 15], axis=1)

In [ ]:
#сделаем так, чтобы не было одинаковых индексов у стран в рейтинге
security_cities_rating[0] = np.array(security_cities_rating.index) + 1

In [6]:
# новые названия для колонок
security_cities_rating = security_cities_rating.rename(columns={1: 'overall_rating', 2: 'overall_score', 
                                                                4: 'digital_sec', 5: 'score1',
                                                                7: 'health_sec', 8: 'score2',
                                                                10: 'infrastucture_sec', 11: 'score3',
                                                                13: 'personal_sec', 14: 'score4',
                                                                16: 'environmental_sec', 17: 'score5'})

In [14]:
#для городов из overall_rating найдем их страны
overall_rating_country = []
geolocator = Nominatim(user_agent="your_project", timeout=3)

for city in tqdm(list(security_cities_rating['overall_rating'])):
    geo_points = [geolocator.geocode(city).raw['lat'], geolocator.geocode(city).raw['lon']]
    overall_rating_country.append(geolocator.reverse(geo_points, language="en").raw['address']['country'])

  0%|          | 0/60 [00:00<?, ?it/s]

In [12]:
#стандартизируем названия стран
coco.logging.getLogger().setLevel(coco.logging.CRITICAL)                                                      
security_cities_rating['overall_rating_country'] = coco.convert(names=overall_rating_country, to='name_short')

ValueError: Length of values (4) does not match length of index (60)

In [ ]:
#переставляем колонки местами
security_cities_rating = security_cities_rating.reindex(columns=[0, 'overall_rating', 'overall_rating_country', 'overall_score', 'digital_sec', 'score1', 'health_sec', 
                                        'score2', 'infrastucture_sec', 'score3', 'personal_sec', 'score4', 'environmental_sec', 'score5'])

### Стандартизация стран


In [ ]:
#приведем в одинаковый вид все страны в таблицах
#используем logging чтобы не получать большое кол-во предупреждений
coco.logging.getLogger().setLevel(coco.logging.CRITICAL)                                                            
human_dev_rating['country'] = coco.convert(names=list(human_dev_rating.country), to='name_short')
quality_of_life_rating['country'] = coco.convert(names=list(quality_of_life_rating.country), to='name_short')
democracy_rating['country'] = coco.convert(names=list(democracy_rating.country), to='name_short')

In [ ]:
#сделаем замену там, где библиотека не смогла выдать ответ
quality_of_life_rating.at[list(quality_of_life_rating['country']).index('not found'), 'country'] = 'Qatar'
quality_of_life_rating.at[list(quality_of_life_rating['country']).index('not found'), 'country'] = 'Egypt'
quality_of_life_rating.at[list(quality_of_life_rating['country']).index(['France', 'Guadeloupe']), 'country'] = 'France'

### Последние штрихи

In [ ]:
print(security_cities_rating.head(3))
print('------------------------------')
print(human_dev_rating.head(3))
print('------------------------------')
print(quality_of_life_rating.head(3))
print('------------------------------')
print(democracy_rating.head(3))

   0 overall_rating overall_rating_country  overall_score digital_sec  score1  \
0  1     Copenhagen                Denmark           82.4      Sydney    83.2   
1  2        Toronto                 Canada           82.2   Singapore    82.8   
2  3      Singapore              Singapore           80.7  Copenhagen    82.2   

  health_sec  score2 infrastucture_sec  score3 personal_sec  score4  \
0      Tokyo    87.7         Hong Kong    93.4   Copenhagen    86.4   
1  Singapore    84.1         Singapore    92.1    Amsterdam    80.5   
2  Hong Kong    84.0        Copenhagen    89.0    Frankfurt    80.3   

  environmental_sec  score5  
0        Wellington    91.7  
1           Toronto    90.3  
2     Washington DC    87.6  
------------------------------
   rank      country    hdi average_annual_growth
0     1  Switzerland  0.962                 0.19%
1     2       Norway  0.961                 0.19%
2     3      Iceland  0.959                 0.56%
------------------------------
   rank 

In [13]:
#сохраняю таблицы в формате csv
security_cities_rating.to_csv('tables/security_cities_rating.csv', index=False)
human_dev_rating.to_csv('tables/human_dev_rating.csv', index=False)
quality_of_life_rating.to_csv('tables/quality_of_life_rating.csv', index=False)
democracy_rating.to_csv('tables/democracy_rating.csv', index=False)